In [1]:
use mavenfuzzyfactory

Commands completed successfully

Total execution time: 00:00:00

**Subiect: Sales Trends**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">We're about to launch a new product, and I'd like to do a&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">deep dive on our current flagship product.&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Can you please pull monthly trends to date for number of&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">sales, total revenue, and total margin generated for the&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">business?</span>

In [2]:
show tables

(6 row(s) affected)

Total execution time: 00:00:01.010

Tables_in_mavenfuzzyfactory
order_item_refunds
order_items
orders
products
website_pageviews
website_sessions


In [3]:
select * from orders limit 3

(3 row(s) affected)

Total execution time: 00:00:01.012

order_id,created_at,website_session_id,user_id,primary_product_id,items_purchased,price_usd,cogs_usd
1,2012-03-19 10:42:46,20,20,1,1,49.99,19.49
2,2012-03-19 19:27:37,104,104,1,1,49.99,19.49
3,2012-03-20 06:44:45,147,147,1,1,49.99,19.49


In [6]:
select YEAR(date(created_at)) as yr, MONTH(DATE(created_at)) as mo,
count(order_id) as sales, sum(price_usd) as revenue, 
sum(price_usd - cogs_usd) as margin
from orders
where created_at < '2013-01-04'
GROUP by 1,2

(11 row(s) affected)

Total execution time: 00:00:01.026

yr,mo,sales,revenue,margin
2012,3,60,2999.40,1830.00
2012,4,99,4949.01,3019.50
2012,5,108,5398.92,3294.00
2012,6,140,6998.60,4270.00
2012,7,169,8448.31,5154.50
2012,8,228,11397.72,6954.00
2012,9,287,14347.13,8753.50
2012,10,371,18546.29,11315.50
2012,11,618,30893.82,18849.00
2012,12,506,25294.94,15433.00


In [32]:
select 
YEAR(date(website_sessions.created_at)) as yr,
MONTH(date(website_sessions.created_at)) as mo,
count(DISTINCT website_sessions.website_session_id) as sessions,
count(DISTINCT orders.order_id) as orders,
count(DISTINCT orders.order_id)/count(website_sessions.website_session_id) as conv_rate,
sum(orders.price_usd)/count(website_sessions.website_session_id) as rev_per_session,
count(DISTINCT case when orders.primary_product_id = 1 then orders.order_id else null end) as prod_1,
count(DISTINCT case when orders.primary_product_id = 2 then orders.order_id else null end) as prod_2
FROM website_sessions
LEFT JOIN orders 
ON website_sessions.website_session_id = orders.website_session_id
where website_sessions.created_at BETWEEN '2012-04-01' AND '2013-04-05'
GROUP BY 1,2

(13 row(s) affected)

Total execution time: 00:00:01.385

yr,mo,sessions,orders,conv_rate,rev_per_session,prod_1,prod_2
2012,4,3734,99,0.0265,1.325391,99,0
2012,5,3736,108,0.0289,1.445107,108,0
2012,6,3963,140,0.0353,1.765985,140,0
2012,7,4249,169,0.0398,1.988305,169,0
2012,8,6097,228,0.0374,1.869398,228,0
2012,9,6546,287,0.0438,2.191740,287,0
2012,10,8183,371,0.0453,2.266441,371,0
2012,11,14011,618,0.0441,2.204969,618,0
2012,12,10072,506,0.0502,2.511412,506,0
2013,1,6401,391,0.0611,3.127025,344,47
